In [1]:
import os
import time
import pandas as pd

In [2]:
#original_dir = os.getcwd()
#print(original_dir)

project_dir = r"c:\Users\Jeff Hwang\Desktop\NBA" # Choose a Path to save your files (such as .ipnyb, .py, .csv)
if not os.path.exists(project_dir):
    print(f"Creating: {project_dir}")
    os.makedirs(project_dir)

os.chdir(project_dir)
print("Working directory:", os.getcwd())

Working directory: c:\Users\Jeff Hwang\Desktop\NBA


In [3]:
subdirectory = os.path.join(project_dir, "csvs")
if not os.path.exists(subdirectory):
    print(f"Creating subdirectory for CSVs: {subdirectory}")
    os.makedirs(subdirectory)

In [4]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = ["2022-23", "2023-24"]
allSeasons = []

for season in seasons:
    gameFinder = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable="Regular Season")
    games = gameFinder.get_data_frames()[0]
    allSeasons.append(games)
    # time.sleep(2)
allSeasonsConcat = pd.concat(allSeasons, ignore_index=True)
#print(allSeasonsConcat)

allSeasonsDir = os.path.join(subdirectory, 'regular_season.csv')
allSeasonsConcat.to_csv(allSeasonsDir, index=False)

In [5]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = ["2021-22", "2022-23", "2023-24"]
teams_abbr = [
    "BOS", "BKN", "NYK", "PHI", "TOR",
    "CHI", "CLE", "DET", "IND", "MIL",
    "ATL", "CHA", "MIA", "ORL", "WAS",
    "DEN", "MIN", "OKC", "POR", "UTA",
    "GSW", "LAC", "LAL", "PHX", "SAC",
    "DAL", "HOU", "MEM", "NOP", "SAS"
]
games = []

for season in seasons:
    gameFinder = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable="Regular Season")
    seasonGames = gameFinder.get_data_frames()[0]
    seasonGames = seasonGames[seasonGames['TEAM_ABBREVIATION'].isin(teams_abbr)]
    games.append(seasonGames)
    time.sleep(2)

games = pd.concat(games, ignore_index=True)
games.to_csv(os.path.join(subdirectory, 'all_games_raw.csv'), index=False)

In [6]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = ["2021-22", "2022-23", "2023-24"]
teams_abbr = [
    "BOS", "BKN", "NYK", "PHI", "TOR",
    "CHI", "CLE", "DET", "IND", "MIL",
    "ATL", "CHA", "MIA", "ORL", "WAS",
    "DEN", "MIN", "OKC", "POR", "UTA",
    "GSW", "LAC", "LAL", "PHX", "SAC",
    "DAL", "HOU", "MEM", "NOP", "SAS"
]

# Directory to save team files
output_dir = "team_files"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Loop through each team and save stats to separate files
for team_abbr in teams_abbr:
    team_games = games[games['TEAM_ABBREVIATION'] == team_abbr]  # Filter data for the team
    team_file_path = os.path.join(output_dir, f"{team_abbr}_games_stats.csv")  # Set file path
    team_games.to_csv(team_file_path, index=False)  # Save to CSV

In [11]:
from nba_api.stats.endpoints import leaguegamefinder

# Directory for input team files
input_dir = "team_files"

# Create a subdirectory for modified files in the same location
output_dir = os.path.join(input_dir, "modified_team_files")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# List of all team files
team_files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]

# Process each team file
for team_file in team_files:
    team_path = os.path.join(input_dir, team_file)
    team_data = pd.read_csv(team_path)
    
    # Ensure games are sorted by date
    if 'GAME_DATE' in team_data.columns:
        team_data['GAME_DATE'] = pd.to_datetime(team_data['GAME_DATE'])
        team_data = team_data.sort_values('GAME_DATE').reset_index(drop=True)
    
    # Columns to calculate rolling averages (exclude non-numeric columns)
    numeric_cols = team_data.select_dtypes(include=['number']).columns
    
    # Create a new DataFrame for modified data
    modified_data = team_data.copy()
    
    # Convert numeric columns to float for compatibility
    modified_data[numeric_cols] = modified_data[numeric_cols].astype(float)
    
    # Replace stats with rolling averages
    for i in range(len(team_data)):
        if i == 0:
            # 1st game: Use the game's own stats
            modified_data.loc[i, numeric_cols] = team_data.loc[i, numeric_cols]
        else:
            # Games 2 onward: Use the average of previous games
            start_idx = max(0, i - 10)  # Take at most 10 previous games
            modified_data.loc[i, numeric_cols] = team_data.loc[start_idx:i-1, numeric_cols].mean()
    
    # Save the modified data
    output_path = os.path.join(output_dir, f"modified_{team_file}")
    modified_data.to_csv(output_path, index=False)

Saved modified file for ATL_games_stats.csv to team_files\modified_team_files\modified_ATL_games_stats.csv
Saved modified file for BKN_games_stats.csv to team_files\modified_team_files\modified_BKN_games_stats.csv
Saved modified file for BOS_games_stats.csv to team_files\modified_team_files\modified_BOS_games_stats.csv
Saved modified file for CHA_games_stats.csv to team_files\modified_team_files\modified_CHA_games_stats.csv
Saved modified file for CHI_games_stats.csv to team_files\modified_team_files\modified_CHI_games_stats.csv
Saved modified file for CLE_games_stats.csv to team_files\modified_team_files\modified_CLE_games_stats.csv
Saved modified file for DAL_games_stats.csv to team_files\modified_team_files\modified_DAL_games_stats.csv
Saved modified file for DEN_games_stats.csv to team_files\modified_team_files\modified_DEN_games_stats.csv
Saved modified file for DET_games_stats.csv to team_files\modified_team_files\modified_DET_games_stats.csv
Saved modified file for GSW_games_sta

In [7]:
allSeasons_OneRow = allSeasonsConcat.groupby("GAME_ID")
allSeasons1RowDF = pd.DataFrame(allSeasons_OneRow)
allSeasonsDir = os.path.join(subdirectory, 'regular_season_one_row.csv')
allSeasons1RowDF.to_csv(allSeasonsDir, index=False)

In [8]:
T1 = allSeasons_OneRow.nth(0).add_prefix("T1_")
T2 = allSeasons_OneRow.nth(1).add_prefix("T2_")

allSeasons1RowDF = pd.DataFrame(T1)
allSeasonsDir = os.path.join(subdirectory, 'regular_season_Team_1.csv')
allSeasons1RowDF.to_csv(allSeasonsDir, index=False)

allSeasons1RowDF = pd.DataFrame(T2)
allSeasonsDir = os.path.join(subdirectory, 'regular_season_Team_2.csv')
allSeasons1RowDF.to_csv(allSeasonsDir, index=False)

In [9]:
mergedData = pd.merge(T1, T2, left_on="T1_GAME_ID", right_on="T2_GAME_ID").reset_index()
mergedDir = os.path.join(subdirectory, 'merged_data.csv')
mergedData.to_csv(mergedDir, index=False)


In [10]:
columns_to_keep = [
    "T1_GAME_ID", "T1_WL", "T1_PTS", "T1_FGM", "T1_FGA", "T1_FG_PCT", "T1_FG3M", "T1_FG3A", 
    "T1_FG3_PCT", "T1_FTM", "T1_FTA", "T1_FT_PCT", "T1_OREB", "T1_DREB", "T1_AST", "T1_TOV", 
    "T1_PF", 
    "T2_WL", "T2_PTS", "T2_FGM", "T2_FGA", "T2_FG_PCT", "T2_FG3M", "T2_FG3A", "T2_FG3_PCT",
    "T2_FTM", "T2_FTA", "T2_FT_PCT", "T2_OREB", "T2_DREB", "T2_AST", "T2_TOV", "T2_PF"
]
filteredData = mergedData[columns_to_keep]
filteredDir = os.path.join(subdirectory, 'filtered_data.csv')
filteredData.to_csv(filteredDir, index=False)